### Pre -work : 


In [ ]:
# Move files around :

rsync -avzhe ssh \
/home/concha-eloko/Téléchargements/Enterobacter_aerogenes_KCTC_2190.fasta \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/post_work

kleborate --kaptive_k \
--kaptive_k_outfile /home/conchae/prediction_depolymerase_tropism/post_work/Enterobacter_aerogenes_KCTC_2190.out \
-a /home/conchae/prediction_depolymerase_tropism/post_work/Enterobacter_aerogenes_KCTC_2190.fasta

***

In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

#import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_2809"

> Open the DF

In [2]:
path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings[0] = dpo_embeddings[0].apply(lambda x : x.split("_Dpo")[0])
dpo_embeddings.set_index([0], inplace = True)


In [3]:
path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings[0] = bea_embeddings[0].apply(lambda x : x.replace("_", "__"))
bea_embeddings.set_index([0], inplace = True)



In [4]:
path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings[0] = towndsend_embeddings[0].apply(lambda x : x.replace("_", "__"))
towndsend_embeddings.set_index([0], inplace = True)


In [5]:
df_embeddings = pd.concat([towndsend_embeddings, bea_embeddings, dpo_embeddings], axis = 0)


> Open df KL types :

In [6]:
path_finetuning = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning"

bea_df = pd.read_csv(f"{path_finetuning}/bea_fine_tuning.df", sep = "\t", header = 0)
bea_df["Protein"] = bea_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_bea = set([kltype.strip() for kltypes in bea_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

ferriol_df = pd.read_csv(f"{path_finetuning}/ferriol_fine_tuning.df", sep = "\t", header = 0)
ferriol_df["Target"] = ferriol_df["Target"].apply(lambda x : x.replace("K", "KL"))
pool_ferriol = set([kltype.strip() for kltypes in ferriol_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

towndsend_df = pd.read_csv(f"{path_finetuning}/towndsend_fine_tuning.df", sep = "\t", header = 0)
towndsend_df["Protein"] = towndsend_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_towndsend = set([kltype.strip() for kltypes in towndsend_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

dico_matrices = {"ferriol" : {"matrix" : ferriol_df, "pool" : pool_ferriol}, 
                 "bea" : {"matrix": bea_df, "pool" : pool_bea}, 
                 "towndsend" : {"matrix" : towndsend_df, "pool" : pool_towndsend}}


In [9]:
others_kltypes = [kltype.strip() for kltypes in bea_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0]
others_kltypes = others_kltypes + [kltype.strip() for kltypes in ferriol_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0]
others_kltypes = others_kltypes + [kltype.strip() for kltypes in towndsend_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0]



Counter(others_kltypes)




Counter({'KL37': 21,
         'KL2': 18,
         'KL22': 17,
         'KL35': 14,
         'KL126': 12,
         'KL61': 11,
         'KL110': 7,
         'KL13': 5,
         'KL140': 5,
         'KL36': 5,
         'KL1': 4,
         'KL24': 4,
         'KL57': 4,
         'KL74': 4,
         'KL5': 4,
         'KL80': 4,
         'KL16': 3,
         'KL30': 3,
         'KL134': 3,
         'KL25': 3,
         'KL54': 3,
         'KL58': 3,
         'KL39': 3,
         'KL71': 3,
         'KL151': 2,
         'KL157': 2,
         'KL20': 2,
         'KL103': 2,
         'KL102': 2,
         'KL149': 2,
         'KL63': 2,
         'KL14': 2,
         'KL10': 2,
         'KL11': 2,
         'KL17': 2,
         'KL26': 2,
         'KL43': 2,
         'KL44': 2,
         'KL45': 2,
         'KL56': 2,
         'KL68': 2,
         'KL18': 2,
         'KL60': 2,
         'KL67': 2,
         'KL82': 2,
         'KL28': 2,
         'KL139': 1,
         'KL62': 1,
         'KL23': 1,
       

> Make the graphs :

In [7]:
def finetune_kltype_df(kltype) :
    positive_lines , negative_lines = [], []
    for author in dico_matrices :
        if kltype in dico_matrices[author]["pool"] : 
            for _,row in dico_matrices[author]["matrix"].iterrows() :
                if row["Target"].count("pass") == 0 :
                    targets = [kltype.strip() for kltype in row["Target"].split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0]
                    if kltype not in targets :
                        negative_lines.append(list(row.values))
                    else :
                        positive_lines.append(list(row.values))
    lines = positive_lines + negative_lines
    n_positives = len(set([item[0] for item in positive_lines]))
    n_negatives = len(set([item[0] for item in negative_lines]))
    df_kltype = pd.DataFrame(lines, columns = ["phage", "depo","KLtypes"])

    return df_kltype , n_positives, n_negatives
    
def build_graph_baseline(df_info, n_positives, n_negatives) : 
    # **************************************************************
    # initialize the graph
    graph_data = HeteroData()
    # Indexation process  
    indexation_nodes_A = [0]
    indexation_nodes_B1 = df_info["phage"].unique().tolist()
    indexation_nodes_B2 = df_info["depo"].unique().tolist() 
    #ID_nodes_A = {item:index for index, item in enumerate(indexation_nodes_A)}
    #ID_nodes_A_r = {index:item for index, item in enumerate(indexation_nodes_A)}
    ID_nodes_B1 = {item:index for index, item in enumerate(indexation_nodes_B1)}
    ID_nodes_B1_r = {index:item for index, item in enumerate(indexation_nodes_B1)}
    ID_nodes_B2 = {item:index for index, item in enumerate(indexation_nodes_B2)}
    ID_nodes_B2_r = {index:item for index, item in enumerate(indexation_nodes_B2)}
    # **************************************************************
    # Make the node feature file : 
    embeddings_columns = [int(i) for i in range(1, 1281)]
    node_feature_A = torch.tensor([0], dtype=torch.float)
    node_feature_B1 = torch.zeros((len(ID_nodes_B1), 0), dtype=torch.float)
    node_feature_B2 = torch.tensor([df_embeddings[df_embeddings.index == depo][embeddings_columns].values[0].tolist() for depo in df_info["depo"]] , dtype=torch.float)
    # feed the graph
    graph_data["A"].x = node_feature_A
    graph_data["B1"].x = node_feature_B1
    graph_data["B2"].x = node_feature_B2
    # **************************************************************
    # Make edge file
    # Node B2 (depolymerase) - Node B1 (prophage) :
    edge_index_B2_B1 = []
    for phage in df_info.phage.unique() :
        all_data_phage = df_info[df_info["phage"] == phage]
        for _, row in all_data_phage.iterrows() :
            edge_index_B2_B1.append([ID_nodes_B2[row["depo"]], ID_nodes_B1[row["phage"]]])
    edge_index_B2_B1 = torch.tensor(edge_index_B2_B1 , dtype=torch.long)
    # feed the graph
    graph_data['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
    # The labels : 
    labels = [1] * n_positives + [0] * n_negatives
    graph_data["B1"].y = torch.tensor(labels)
    # Training fraction :
    train_mask = [1]* len(labels)
    graph_data["B1"].train_mask = torch.tensor(train_mask)
    return graph_data 
    


In [33]:
test_df, n_pos, n_neg = finetune_kltype_df("KL20")
graph_test = build_graph_baseline(test_df , n_pos , n_neg)

In [34]:
Counter(graph_test["B1"].y.tolist())

Counter({0: 34, 1: 1})

In [22]:
graph_test

HeteroData(
  A={ x=[1] },
  B1={
    x=[84, 0],
    y=[84],
    train_mask=[84]
  },
  B2={ x=[134, 1280] },
  (B2, expressed, B1)={ edge_index=[2, 134] }
)

***

In [ ]:
def train_graph(KL_type) :
    checkpoint_path = f"{path_ensemble}/{KL_type}.finetuned.TropiGATv2.2211.pt"
    with open(f"{path_work}/train_nn/fintuning_files/{KL_type}__finetuned_node_classification.2111.log" , "w") as log_outfile :
        n_prophage = dico_prophage_count[KL_type]
        graph_data_kltype = graph_dico[KL_type]
        if n_prophage <= 125 : 
            model = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)
            n = "small"
        else : 
            model = TropiGAT_models.TropiGAT_big_module(hidden_channels = 1280, heads = 1)
            n = "big"
        model.load_state_dict(torch.load(f"{path_ensemble}/{KL_type}.TropiGATv2.2809.pt"))
        model(graph_data_kltype)
        # 
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001 , weight_decay= 0.000001)
        scheduler = ReduceLROnPlateau(optimizer, 'min')
        criterion = torch.nn.BCEWithLogitsLoss()
        #
        try : 
            for epoch in range(50):
                train_loss = TropiGAT_models.train(model, graph_data_kltype, optimizer,criterion)
                # Save the model checkpoint
                torch.save(model.state_dict(), checkpoint_path)
                print(f"Checkpoint saved: {checkpoint_path}")
        except Exception as e :
            log_outfile.write(f"***Issue here : {e}")



